<a href="https://colab.research.google.com/github/AadityaPanchakshikar/Technical-Analysis-w-py/blob/main/Pattern%2BScanner_Custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pattern Scanner 1

In [ ]:
import talib
import numpy as np
import itertools
import operator
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
import math
warnings.filterwarnings('ignore')

In [ ]:
candle_names = ['3LINESTRIKE','3BLACKCROWS','EVENINGSTAR','HAMMER','HANGINGMAN','HARAMI','HARAMICROSS','ENGULFING','SHOOTINGSTAR','MARUBOZU','DOJI']
candle_names

['3LINESTRIKE',
 '3BLACKCROWS',
 'EVENINGSTAR',
 'HAMMER',
 'HANGINGMAN',
 'HARAMI',
 'HARAMICROSS',
 'ENGULFING',
 'SHOOTINGSTAR',
 'MARUBOZU',
 'DOJI']

In [ ]:
df = pd.read_csv("nifty5min.csv")
df = df[(df.Date > '2022-09-02 9:00:00') & (df.Date < '2022-09-06 15:30:00')]
df.set_index("Date", inplace=True)
df = df[['Open', 'High', 'Low', 'Close']]
df.head(10)

,Open,High,Low,Close
Date,,,,
2022-09-05 09:15:00,17551,17616,17551,17612
2022-09-05 09:20:00,17613,17615,17584,17597
2022-09-05 09:25:00,17601,17608,17571,17595
2022-09-05 09:30:00,17595,17632,17592,17632
2022-09-05 09:35:00,17631,17639,17620,17627
2022-09-05 09:40:00,17627,17646,17625,17638
2022-09-05 09:45:00,17639,17641,17620,17622
2022-09-05 09:50:00,17622,17639,17609,17638
2022-09-05 09:55:00,17639,17644,17630,17635


Get Trendline

In [ ]:
real = talib.HT_TRENDLINE(df.Close)
df['trend'] = real
df.dropna(inplace=True)

Get Slope

In [ ]:
df2 = df[['trend']]
df2['deg'] = 0
df2.reset_index(inplace=True)
i = 3
while i in range(3, len(df2)):
    x = df2.index[i-3:i]
    y = df2.trend[i-3:i]
    slope, intercept = np.polyfit(x,y,1)
    deg = math.degrees(slope)
    df2['deg'].iat[i] = deg
    i = i + 1
df2.set_index('Date', inplace=True)
df['deg'] = df2['deg']
df

,Open,High,Low,Close,trend,deg
Date,,,,,,
2022-09-05 14:45:00,17670,17671,17662,17664,17656.729781,0.000000
2022-09-05 14:50:00,17663,17668,17659,17663,17657.307932,0.000000
2022-09-05 14:55:00,17662,17672,17653,17654,17657.611154,0.000000
2022-09-05 15:00:00,17654,17678,17651,17677,17657.711009,25.249460
2022-09-05 15:05:00,17677,17679,17665,17670,17657.868213,11.547296
...,...,...,...,...,...,...
2022-09-06 15:05:00,17661,17681,17658,17671,17682.404821,-125.540386
2022-09-06 15:10:00,17672,17674,17651,17653,17679.760167,-127.036502
2022-09-06 15:15:00,17653,17657,17649,17652,17677.481170,-143.167693


Determine whether Bullish or Bearish

In [ ]:
df['diff'] = np.where(df.deg < 0,-1,1)
df

,Open,High,Low,Close,trend,deg,diff
Date,,,,,,,
2022-09-05 14:45:00,17670,17671,17662,17664,17656.729781,0.000000,1
2022-09-05 14:50:00,17663,17668,17659,17663,17657.307932,0.000000,1
2022-09-05 14:55:00,17662,17672,17653,17654,17657.611154,0.000000,1
2022-09-05 15:00:00,17654,17678,17651,17677,17657.711009,25.249460,1
2022-09-05 15:05:00,17677,17679,17665,17670,17657.868213,11.547296,1
...,...,...,...,...,...,...,...
2022-09-06 15:05:00,17661,17681,17658,17671,17682.404821,-125.540386,-1
2022-09-06 15:10:00,17672,17674,17651,17653,17679.760167,-127.036502,-1
2022-09-06 15:15:00,17653,17657,17649,17652,17677.481170,-143.167693,-1


In [ ]:
for candle in candle_names:
    df[candle] = getattr(talib, 'CDL' + candle)(df['Open'], df['High'], df['Low'], df['Close'])
df.head()

,Open,High,Low,Close,trend,deg,diff,3LINESTRIKE,3BLACKCROWS,EVENINGSTAR,HAMMER,HANGINGMAN,HARAMI,HARAMICROSS,ENGULFING,SHOOTINGSTAR,MARUBOZU,DOJI
Date,,,,,,,,,,,,,,,,,,
2022-09-05 14:45:00,17670,17671,17662,17664,17656.729781,0.000000,1,0,0,0,0,0,0,0,0,0,0,0
2022-09-05 14:50:00,17663,17668,17659,17663,17657.307932,0.000000,1,0,0,0,0,0,0,0,0,0,0,0
2022-09-05 14:55:00,17662,17672,17653,17654,17657.611154,0.000000,1,0,0,0,0,0,0,0,0,0,0,0
2022-09-05 15:00:00,17654,17678,17651,17677,17657.711009,25.249460,1,0,0,0,0,0,0,0,0,0,0,0
2022-09-05 15:05:00,17677,17679,17665,17670,17657.868213,11.547296,1,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df.sample(20)

,Open,High,Low,Close,trend,deg,diff,3LINESTRIKE,3BLACKCROWS,EVENINGSTAR,HAMMER,HANGINGMAN,HARAMI,HARAMICROSS,ENGULFING,SHOOTINGSTAR,MARUBOZU,DOJI
Date,,,,,,,,,,,,,,,,,,
2022-09-06 14:10:00,17692,17698,17691,17697,17695.515385,66.197088,1,0,0,0,0,0,0,0,0,0,0,0
2022-09-06 11:25:00,17656,17664,17652,17655,17646.640057,-30.125795,-1,0,0,0,0,0,0,0,0,0,0,100
2022-09-06 13:55:00,17683,17699,17681,17696,17692.466931,35.849284,1,0,0,0,0,0,0,0,0,0,0,0
2022-09-06 13:40:00,17693,17696,17688,17694,17689.826799,37.931637,1,0,0,0,0,0,0,0,0,0,0,100
2022-09-06 13:30:00,17698,17714,17698,17711,17688.987859,43.897494,1,0,0,0,0,0,0,0,0,0,0,0
2022-09-05 15:20:00,17667,17668,17658,17660,17658.741749,17.659525,1,0,0,0,0,0,0,0,0,0,0,0
2022-09-06 13:50:00,17679,17691,17674,17684,17691.078175,22.862922,1,0,0,0,0,0,0,0,0,0,0,0
2022-09-06 14:55:00,17662,17664,17650,17655,17686.839232,-142.986693,-1,0,0,0,0,0,0,0,0,0,0,0
2022-09-05 15:05:00,17677,17679,17665,17670,17657.868213,11.547296,1,0,0,0,0,0,0,0,0,0,0,0


Extract 5 top patterns found in every candle

In [ ]:
df['Pattern1'] = np.nan
df['Pattern2'] = np.nan
df['Pattern3'] = np.nan
df['Pattern4'] = np.nan
df['Pattern5'] = np.nan
df['Patterns'] = np.nan

In [ ]:
for index, row in df.iterrows():
    #print('index', index)
    # no pattern found
    if len(row[candle_names]) - sum(row[candle_names] == 0) == 0:
        pass
    # single pattern found
    elif len(row[candle_names]) - sum(row[candle_names] == 0) == 1:
        # bull pattern 100 or 200
        if any(row[candle_names].values > 0):
            pattern = list(itertools.compress(row[candle_names].keys(), row[candle_names].values != 0))[0] + '_Bull'
            #print(index, pattern)
            #if df.loc[df.index==index, 'diff'][0] == -1:
            if df.loc[df.index==index, 'deg'][0] < 0:
                print('pattern:', pattern)
                df.loc[df.index==index, 'Pattern1'] = pattern
                df.loc[df.index==index, 'Patterns'] = pattern
        # bear pattern -100 or -200
        else:
            pattern = list(itertools.compress(row[candle_names].keys(), row[candle_names].values != 0))[0] + '_Bear'
            #if df.loc[df.index==index, 'diff'][0] == 1:
            if df.loc[df.index==index, 'deg'][0] > 0:
                print('pattern:', pattern)
                df.loc[df.index==index, 'Pattern1'] = pattern
                df.loc[df.index==index, 'Patterns'] = pattern
            #print(index, pattern)
    # multiple patterns matched -- select best performance
    else:
        # filter out pattern names from bool list of values
        patterns = list(itertools.compress(row[candle_names].keys(), row[candle_names].values != 0))
        container = []
        i = 0
        for pattern in patterns:
            if row[pattern] > 0:
                #if df.loc[df.index==index, 'diff'][0] == -1:
                if df.loc[df.index==index, 'deg'][0] < 0:
                    p = pattern + '_Bull'
                    container.append(p)
                    if i == 0:
                        df.loc[df.index==index, 'Pattern1'] = p
                    elif i == 1:
                        df.loc[df.index==index, 'Pattern2'] = p
                    elif i == 2:
                        df.loc[df.index==index, 'Pattern3'] = p
                    elif i == 3:
                        df.loc[df.index==index, 'Pattern4'] = p
                    elif i == 4:
                        df.loc[df.index==index, 'Pattern5'] = p
            else:
                #if df.loc[df.index==index, 'diff'][0] == 1:
                if df.loc[df.index==index, 'deg'][0] > 0:
                    print('if 4')
                    p = pattern + '_Bear'
                    container.append(p)
                    if i == 0:
                        df.loc[df.index==index, 'Pattern1'] = p
                    elif i == 1:
                        df.loc[df.index==index, 'Pattern2'] = p
                    elif i == 2:
                        df.loc[df.index==index, 'Pattern3'] = p
                    elif i == 3:
                        df.loc[df.index==index, 'Pattern4'] = p
                    elif i == 4:
                        df.loc[df.index==index, 'Pattern5'] = p
            i += 1
        print(index)
        if len(container) > 0:
            print('container', container)
            df.loc[df.index==index, 'Patterns'] = ",".join(container)#str(container)
        print('Patterns', *patterns)

pattern: ENGULFING_Bull
2022-09-06 11:00:00
container ['DOJI_Bull']
Patterns HANGINGMAN DOJI
2022-09-06 11:10:00
container ['DOJI_Bull']
Patterns HANGINGMAN DOJI
pattern: DOJI_Bull
pattern: MARUBOZU_Bull
pattern: DOJI_Bull
pattern: MARUBOZU_Bull
pattern: MARUBOZU_Bear
2022-09-06 12:15:00
Patterns ENGULFING MARUBOZU
pattern: ENGULFING_Bear
pattern: DOJI_Bull
2022-09-06 15:10:00
Patterns ENGULFING MARUBOZU
pattern: DOJI_Bull
2022-09-06 15:20:00
container ['HAMMER_Bull', 'DOJI_Bull']
Patterns HAMMER DOJI
2022-09-06 15:25:00
container ['HAMMER_Bull', 'DOJI_Bull']
Patterns HAMMER HANGINGMAN DOJI


In [ ]:
df2 = df[df['Patterns'].notnull()]
df2[['Close','trend','diff', 'Pattern1', 'Pattern2', 'Pattern3', 'Pattern4', 'Pattern5', 'Patterns']]

,Close,trend,diff,Pattern1,Pattern2,Pattern3,Pattern4,Pattern5,Patterns
Date,,,,,,,,,
2022-09-06 10:35:00,17609,17653.239677,-1,ENGULFING_Bull,NaN,NaN,NaN,NaN,ENGULFING_Bull
2022-09-06 11:00:00,17635,17648.621720,-1,NaN,DOJI_Bull,NaN,NaN,NaN,DOJI_Bull
2022-09-06 11:10:00,17667,17648.533474,-1,NaN,DOJI_Bull,NaN,NaN,NaN,DOJI_Bull
2022-09-06 11:25:00,17655,17646.640057,-1,DOJI_Bull,NaN,NaN,NaN,NaN,DOJI_Bull
2022-09-06 11:40:00,17673,17643.668074,-1,MARUBOZU_Bull,NaN,NaN,NaN,NaN,MARUBOZU_Bull
2022-09-06 11:50:00,17669,17642.101261,-1,DOJI_Bull,NaN,NaN,NaN,NaN,DOJI_Bull
2022-09-06 11:55:00,17691,17642.167342,-1,MARUBOZU_Bull,NaN,NaN,NaN,NaN,MARUBOZU_Bull
2022-09-06 12:10:00,17683,17648.115152,1,MARUBOZU_Bear,NaN,NaN,NaN,NaN,MARUBOZU_Bear
2022-09-06 14:15:00,17688,17695.837464,1,ENGULFING_Bear,NaN,NaN,NaN,NaN,ENGULFING_Bear


To display pattern marker below Low price of candle

In [ ]:
df['Pattern_Exist'] = np.where(df['Patterns'].notnull(), df.Low,np.nan)
df.tail(10)

,Open,High,Low,Close,trend,deg,diff,3LINESTRIKE,3BLACKCROWS,EVENINGSTAR,...,SHOOTINGSTAR,MARUBOZU,DOJI,Pattern1,Pattern2,Pattern3,Pattern4,Pattern5,Patterns,Pattern_Exist
Date,,,,,,,,,,,,,,,,,,,,,
2022-09-06 14:40:00,17665,17669,17639,17641,17694.131026,-13.979517,-1,0,0,0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-06 14:45:00,17641,17667,17638,17662,17691.730192,-67.874278,-1,0,0,0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-06 14:50:00,17661,17663,17643,17662,17689.139848,-118.711019,-1,0,0,0,...,0,0,100,DOJI_Bull,NaN,NaN,NaN,NaN,DOJI_Bull,17643.0
2022-09-06 14:55:00,17662,17664,17650,17655,17686.839232,-142.986693,-1,0,0,0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-06 15:00:00,17654,17668,17649,17661,17684.757662,-140.115703,-1,0,0,0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-06 15:05:00,17661,17681,17658,17671,17682.404821,-125.540386,-1,0,0,0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-06 15:10:00,17672,17674,17651,17653,17679.760167,-127.036502,-1,0,0,0,...,0,-100,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-06 15:15:00,17653,17657,17649,17652,17677.481170,-143.167693,-1,0,0,0,...,0,0,100,DOJI_Bull,NaN,NaN,NaN,NaN,DOJI_Bull,17649.0
2022-09-06 15:20:00,17651,17653,17647,17650,17675.383918,-141.052227,-1,0,0,0,...,0,0,100,HAMMER_Bull,DOJI_Bull,NaN,NaN,NaN,"HAMMER_Bull,DOJI_Bull",17647.0


In [ ]:
x = df.index.tolist()
z = df.Patterns.tolist()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Candlestick(x=df.index,
                open=df.Open, high=df.High,
                low=df.Low, close=df.Close))

fig.update_layout(xaxis_rangeslider_visible=False)
fig.update_layout(
    width=800, height=600,
    title="Nifty",
    yaxis_title='Price',
    xaxis=dict(type = "category")
)

fig.add_trace(go.Scatter(
    x=x,
    y=df.trend,
    name ='Trendline',
    mode="markers",
    marker_size=10,
    marker = dict(color='green', symbol = "square-dot")
))

fig.add_trace(go.Scatter(
    x=x,
    y=df.Pattern_Exist,
    customdata=z,
    name ='Pattern',
    mode="markers",
    marker_size=10,
    marker = dict(color='blue', symbol = "triangle-down"), hovertemplate='%{customdata}'
))

y = df['diff'].tolist()
#y = df['deg'].tolist()
fig.add_trace(go.Scatter(x=df.index, y=y, name="Trend"),
secondary_y=True,)

fig.show()